## Name: Local Trees with Support Vector Sets
### Date: 20/05/2025
### Status: Pending results
### Idea: 
The idea is the following given the data set for it's training sample find S different balanced subsets of the training data of containing M training samples from each class, which when fitting a decision tree classifier on each one of them the dt correctly predicts the sample at hand. This process is repeated for all the training samples in the data and the idea is to keep track of the possible good-subsets per sample, lets call them Support vector sets (SVSs). Then at inference time for the query  sample we find it's most closest K training samples and we retrieve the corresponding SVS's. Then we fit K*S different decision trees on these different SVSs and and the final decision is made through voting (probably weighted). 

Implementation wise I think of two strategies finding the support Vector set for it s training sample completely at random or by sampling based on the distance of it's training sample to each other in the data set. 

### Details:


### Results:
 Works a bit. Best one is better on 25/93, worse 16/93, equal on the rest against a standardized LR.

In [76]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.model_selection import train_test_split
import numpy as np
import random


# class LTSVS(BaseEstimator, ClassifierMixin):
#     def __init__(self, S=2, M=2, K=2, sampling_strategy='random', random_state=None):
#         self.S = S
#         self.M = M
#         self.K = K
#         self.sampling_strategy = sampling_strategy
#         self.random_state = random_state

#     def fit(self, X, y):
#         X, y = check_X_y(X, y)
#         self.X_train_ = X
#         self.y_train_ = y
#         self.classes_ = np.unique(y)
#         rng = np.random.RandomState(self.random_state)
#         self.svs_dict_ = {}  # Support vector sets per sample index

#         for idx in range(len(X)):
#             x_i, y_i = X[idx], y[idx]
#             sv_sets = []

#             attempts = 0
#             while len(sv_sets) < self.S and attempts < self.S * 10:
#                 if self.sampling_strategy == 'random':
#                     # Balanced subset
#                     pos_idx = np.where(y == self.classes_[0])[0]
#                     neg_idx = np.where(y == self.classes_[1])[0]

#                     pos_sample = rng.choice(pos_idx, min(self.M, len(pos_idx)), replace=False)
#                     neg_sample = rng.choice(neg_idx, min(self.M, len(neg_idx)), replace=False)
#                     subset_idx = np.concatenate([pos_sample, neg_sample])
#                 else:  # distance-based
#                     dists = np.linalg.norm(X - x_i, axis=1)
#                     nearest_idx = np.argsort(dists)[1:2*self.M+1]
#                     class_0 = [i for i in nearest_idx if y[i] == self.classes_[0]]
#                     class_1 = [i for i in nearest_idx if y[i] == self.classes_[1]]
#                     if len(class_0) < self.M or len(class_1) < self.M:
#                         attempts += 1
#                         continue
#                     subset_idx = np.array(class_0[:self.M] + class_1[:self.M])

#                 clf = DecisionTreeClassifier(random_state=self.random_state)
#                 clf.fit(X[subset_idx], y[subset_idx])
#                 if clf.predict(x_i.reshape(1, -1))[0] == y_i:
#                     sv_sets.append(subset_idx)
#                 attempts += 1

#             self.svs_dict_[idx] = sv_sets
#         return self

#     def predict(self, X):
#         check_is_fitted(self)
#         X = check_array(X)
#         preds = []

#         nbrs = NearestNeighbors(n_neighbors=self.K).fit(self.X_train_)
#         _, indices = nbrs.kneighbors(X)

#         for i in range(len(X)):
#             trees = []
#             for neighbor in indices[i]:
#                 sv_sets = self.svs_dict_.get(neighbor, [])
#                 for sv in sv_sets:
#                     clf = DecisionTreeClassifier(random_state=self.random_state)
#                     clf.fit(self.X_train_[sv], self.y_train_[sv])
#                     trees.append(clf)

#             if not trees:
#                 print(f"SHOULD NOT BE HERE?")
#                 preds.append(random.choice(self.classes_))  # fallback
#             else:
#                 votes = [tree.predict(X[i].reshape(1, -1))[0] for tree in trees]
#                 preds.append(max(set(votes), key=votes.count))

#         return np.array(preds)
    
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances 
from scipy.special import softmax
from sklearn.preprocessing import StandardScaler
eps = 1e-3

def argsmallest_n(a, n):
    ret = np.argpartition(a, n)[:n]
    b = np.take(a, ret)
    return np.take(ret, np.argsort(b))

class LPSVS(BaseEstimator, ClassifierMixin):
    def __init__(self, num_neigh=5, num_sets_per_sample=10, random_state=42, project=100):
        self.num_sets_per_sample = num_sets_per_sample
        self.num_neigh = num_neigh
        self.random_state = random_state
        self.project = project
            

    def fit(self, X, y):
        X, y = check_X_y(X, y)
        if self.project > 0:
            n_components = min(self.project, X.shape[1])
            self.projector = PCA(n_components=n_components, random_state=self.random_state)
            X = self.projector.fit_transform(X)
        self.normalize = StandardScaler()
        X= self.normalize.fit_transform(X)
        self.X_train_ = X
        self.y_train_ = y
        self.classes_ = np.unique(y)
        self.svs_dict_ = {}
        rng = np.random.RandomState(self.random_state)
        dists = pairwise_distances(X) + eps
        np.fill_diagonal(dists, np.max(dists))

        for idx, x_i in enumerate(X):
            y_i = y[idx]
            sv_sets = []
            attempts = 0
            cur_dist = dists[idx,:]
            # cur_proba = (1/cur_dist) / cur_dist.sum()

            while len(sv_sets) < self.num_sets_per_sample and attempts < self.num_sets_per_sample * 10:
                idx_0 = np.where(y != y_i)[0]
                idx_1 = np.where(y == y_i)[0]

                if len(idx_0) < self.num_neigh or len(idx_1) < self.num_neigh:
                    break

                sample_0 = rng.choice(idx_0, min(self.num_neigh,softmax(1/cur_dist[idx_0]).nonzero()[0].shape[0]), replace=False, p=softmax(1/cur_dist[idx_0]))
                sample_1 = rng.choice(idx_1, min(self.num_neigh,softmax(1/cur_dist[idx_1]).nonzero()[0].shape[0]), replace=True, p=softmax(1/cur_dist[idx_1]))
                subset_idx = np.concatenate([sample_0, sample_1])

                X_subset = X[subset_idx]
                y_subset = y[subset_idx]

                class0 = X_subset[y_subset != y_i]
                class1 = X_subset[y_subset == y_i]

                centroid_0 = np.mean(class0, axis=0)
                centroid_1 = np.mean(class1, axis=0)
                w = centroid_1 - centroid_0

                if np.dot(x_i - (centroid_0 + centroid_1) / 2, w) > 0:
                    sv_sets.append((centroid_0, centroid_1, y_i))

                attempts += 1

            self.svs_dict_[idx] = sv_sets
        return self

    def predict(self, X):
        check_is_fitted(self)
        X = check_array(X)
        if self.project > 0:
            X = self.projector.transform(X)
        X = self.normalize.transform(X)
        preds = []

        nbrs = NearestNeighbors(n_neighbors=self.num_neigh).fit(self.X_train_)
        dists, indices = nbrs.kneighbors(X)
        dists += eps
        weights = (1/dists) / ((1/dists).sum(axis=1).reshape(-1,1))

        for i in range(len(X)):
            cur_pred = np.zeros_like(self.classes_, dtype=float)
            for neighbor_index, neighbor in enumerate(indices[i]):
                cur_weight = weights[i, neighbor_index]
                sv_sets = self.svs_dict_.get(neighbor, [])
                #print(f'test index {i} \t neig indeex {neighbor_index} \t neigh sample index/label {neighbor},{self.y_train_[neighbor]} \t neigh dist {cur_weight}')
                for svs_index, (centroid_0, centroid_1, y_i) in enumerate(sv_sets):
                    w = centroid_1 - centroid_0
                    dist = np.dot(X[i] - (centroid_0 + centroid_1) / 2, w)
                    sign = int(dist > 0)
                    same_sign_and_label = int(sign == y_i )
                    #print(f'svs index {svs_index} \t dist {dist} \t sign {sign} \t will add to {same_sign_and_label} \t proba {cur_weight}')
                    cur_pred[same_sign_and_label] += 1*cur_weight
                    #print(f'Updated cur pred {cur_pred}')
            #print(cur_pred)
            preds.append(cur_pred.argmax())

        # print(f'PRED', np.array(preds))
        return np.array(preds)
    
clf = LPSVS(num_neigh=5, num_sets_per_sample=1, project=-1)
clf.fit(X_train, y_train)
clf.predict(X_test)
    

array([0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [98]:
import pandas as pd
import cached_path
from pmlb import fetch_data
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.model_selection import StratifiedKFold, cross_val_predict
import time
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
from scipy.special import softmax
from sklearn.base import clone

path_to_data_summary = "https://raw.githubusercontent.com/EpistasisLab/pmlb/master/pmlb/all_summary_stats.tsv"
dataset_df = pd.read_csv(cached_path.cached_path(path_to_data_summary), sep="\t")

classification_datasets = dataset_df[
    # (dataset_df["n_binary_features"] == dataset_df["n_features"])
    (dataset_df["task"] == "classification")
    & (dataset_df["n_classes"] == 2)
    & (dataset_df["n_features"] <= 150)
    # & (dataset_df["n_features"] >= 10)
    & (dataset_df["n_instances"] <= 1000)
]["dataset"][:]

print(len(classification_datasets))

models = {
    "Baseline": {},
    # "LTSVS_random": {"sampling_strategy":"random"},
    "LPSVS_No_11_1": {'project':-1, "num_neigh":11, "num_sets_per_sample":1},
    "LPSVS_No_5_1": {'project':-1, "num_neigh":5, "num_sets_per_sample":1},
    "LPSVS_No_5_5": {'project':-1, "num_neigh":5, "num_sets_per_sample":5},
    "LPSVS_No_11_5": {'project':-1, "num_neigh":11, "num_sets_per_sample":5},
    "LPSVS_100_11_1": {'project':100, "num_neigh":11, "num_sets_per_sample":1},
    "LPSVS_100_5_1": {'project':100, "num_neigh":5, "num_sets_per_sample":1},
    "LPSVS_100_5_5": {'project':100, "num_neigh":5, "num_sets_per_sample":5},
    "LPSVS_100_11_5": {'project':100, "num_neigh":11, "num_sets_per_sample":5},
    # "LTSVS_random2": {"sampling_strategy":"random", "S":100},
    # "LTSVS_random2": {"sampling_strategy":"random", "S":100},
    #"LTSVS_distance": {"sampling_strategy":"distance"},
    #"Imp_LTSVS": {},
    # "CRH_pc80":{"top_k_to_keep":0.8},
    # "CRH_pc90":{"top_k_to_keep":0.9},

}


number_of_cv_folds = 5
random_state = 42

cv = StratifiedKFold(number_of_cv_folds, random_state=random_state, shuffle=True)
base_class = Pipeline([
    ('st', StandardScaler()),
    ('clf', LogisticRegression(random_state=42))
    ])   #RandomForestClassifier(random_state=42)#DecisionTreeClassifier(max_depth=None, random_state=42)#

res = [] 
for dataset_index, classification_dataset in enumerate(classification_datasets[::-1][:]):
    
    print(f"{classification_dataset} ({dataset_index + 1}/{len(classification_datasets) + 1})")
    if 'deprecated' in classification_dataset:
        print(f"Skipping {classification_dataset} as deprecated from PMLB...")
        continue
    try:
        X, y = fetch_data(classification_dataset, return_X_y=True)
    except ValueError as e:
        print(f'Probably not found dataset {classification_dataset} in PMLB and skipping...\n {e}')
        continue
    if y.max() != 1 or y.min() != 0:
        for wanted, actual in enumerate(np.unique(y)):
            y[y==actual] = wanted
        
    imb_ratio = np.bincount(y).max() / np.bincount(y).min()
    print(f"{X.shape} with ratio : {imb_ratio:.4f}\n")
    

    for model_name, model_kwargs in models.items():
        y_pred = np.empty_like(y)
        sample_weights = None
        time_s = time.time()
        for train_indices, test_indices in cv.split(X,y):
            X_train, y_train = X[train_indices], y[train_indices]
            X_test, y_test = X[test_indices], y[test_indices]
            
            X_train_filtered = X_train.copy()
            y_train_filtered = y_train.copy()
            if model_name.startswith("LPSVS"):
                clf = LPSVS(**model_kwargs)
            else:
                clf = clone(base_class)
            #print(model_name, X_train_filtered.shape[0])
            clf.fit(X_train_filtered , y_train_filtered)
            y_pred_cur = clf.predict(X_test)

            y_pred[test_indices] = y_pred_cur
            #print(f'TRUE', y_test)
            
        
        
        acc = accuracy_score(y, y_pred)
        (prec, rec, f1, sup) = precision_recall_fscore_support(
            y, y_pred, average="binary"
        )
            
        
        print(model_name)    
        print(classification_report(y, y_pred))
        time_end = time.time() - time_s

        res.append((classification_dataset, imb_ratio, model_name, time_end, acc, prec, rec, f1))
        
res = pd.DataFrame(res, columns=['dataset', 'dataset_class_imb', 'model', 'time', 'acc', 'pr', 'rec', 'f1'])

# Step 2: Sort each group by 'f1'
sorted_df = res.groupby('dataset').apply(lambda x: x.sort_values(by='f1', ascending=False)).reset_index(drop=True)

# Step 3: Assign ranks within each group
sorted_df['rank'] = sorted_df.groupby('dataset').cumcount() + 1

# Step 4: Calculate mean rank for each model across all datasets
mean_ranks = sorted_df.groupby('model')['rank'].mean().reset_index().sort_values(by='rank')

print(mean_ranks)
            

73
xd6 (1/74)
(973, 9) with ratio : 2.0217

Baseline
              precision    recall  f1-score   support

           0       0.82      0.92      0.87       651
           1       0.79      0.60      0.68       322

    accuracy                           0.81       973
   macro avg       0.80      0.76      0.78       973
weighted avg       0.81      0.81      0.81       973

LPSVS_No_11_1
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       651
           1       0.99      0.99      0.99       322

    accuracy                           1.00       973
   macro avg       1.00      1.00      1.00       973
weighted avg       1.00      1.00      1.00       973

LPSVS_No_5_1
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       651
           1       0.94      0.95      0.95       322

    accuracy                           0.96       973
   macro avg       0.96      0.96      0.96     

/tmp/ipykernel_1693461/1944496001.py:119: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sorted_df = res.groupby('dataset').apply(lambda x: x.sort_values(by='f1', ascending=False)).reset_index(drop=True)


In [99]:
sorted_df.groupby('model')['time'].mean().reset_index().sort_values(by='time')

,model,time
0,Baseline,0.070458
7,LPSVS_No_5_1,0.577975
5,LPSVS_No_11_1,0.628548
3,LPSVS_100_5_1,0.728710
1,LPSVS_100_11_1,0.750811
8,LPSVS_No_5_5,2.735923
4,LPSVS_100_5_5,2.878917
2,LPSVS_100_11_5,2.985577
6,LPSVS_No_11_5,3.012182


In [100]:
sorted_df

,dataset,dataset_class_imb,model,time,acc,pr,rec,f1,rank
0,analcatdata_aids,1.000000,Baseline,0.047709,0.640000,0.733333,0.440000,0.550000,1
1,analcatdata_aids,1.000000,LPSVS_No_11_1,0.170789,0.400000,0.407407,0.440000,0.423077,2
2,analcatdata_aids,1.000000,LPSVS_No_5_5,0.629542,0.400000,0.407407,0.440000,0.423077,3
3,analcatdata_aids,1.000000,LPSVS_No_11_5,0.656605,0.380000,0.392857,0.440000,0.415094,4
4,analcatdata_aids,1.000000,LPSVS_100_11_5,0.697906,0.420000,0.416667,0.400000,0.408163,5
...,...,...,...,...,...,...,...,...,...
373,xd6,2.021739,LPSVS_100_5_1,1.492519,0.979445,0.977848,0.959627,0.968652,5
374,xd6,2.021739,LPSVS_100_5_5,6.675687,0.978417,0.980831,0.953416,0.966929,6
375,xd6,2.021739,LPSVS_No_5_5,6.762811,0.973279,0.974359,0.944099,0.958991,7
376,xd6,2.021739,LPSVS_No_5_1,1.477298,0.964029,0.944272,0.947205,0.945736,8


In [101]:
model_names = res['model'].unique()
wins_score = np.zeros((len(model_names), len(model_names)))
metric_to_score = 'f1'
for classification_dataset in res['dataset'].unique():
    cur_df = res[res['dataset'] == classification_dataset]
    # print(classification_dataset)
    # print(cur_df.sort_values('f1', ascending=False)[['model', 'time', 'acc', 'f1']])
    # print()
    cur_df = cur_df.set_index('model')
    score_metric = cur_df[metric_to_score]
    for i, m1 in enumerate(model_names):
        for j, m2 in enumerate(model_names[i:]):
            if cur_df.loc[m1][metric_to_score] > cur_df.loc[m2][metric_to_score]:
                wins_score[i, j+i] += 1
            elif cur_df.loc[m1][metric_to_score] < cur_df.loc[m2][metric_to_score]:
                wins_score[j+i, i] += 1
            else:
                pass
order_of_models = wins_score.mean(axis=1).argsort()[::-1]
wins_score = wins_score[order_of_models, :][:, order_of_models]
# Uncomment this for percentage wins
# wins_score /= res['dataset'].unique().shape[0]
print('WINS')
print(pd.DataFrame(wins_score, columns = model_names[order_of_models], index=model_names[order_of_models]))

WINS
                LPSVS_No_5_5  LPSVS_No_11_5  ...  LPSVS_100_11_1  LPSVS_100_5_1
LPSVS_No_5_5             0.0           24.0  ...            28.0           30.0
LPSVS_No_11_5           16.0            0.0  ...            29.0           30.0
LPSVS_No_5_1            14.0           21.0  ...            27.0           26.0
LPSVS_No_11_1           17.0           16.0  ...            24.0           28.0
LPSVS_100_11_5          14.0           10.0  ...            26.0           28.0
LPSVS_100_5_5           13.0           17.0  ...            25.0           24.0
Baseline                13.0           12.0  ...            19.0           20.0
LPSVS_100_11_1          13.0           13.0  ...             0.0           25.0
LPSVS_100_5_1           11.0           10.0  ...            17.0            0.0

[9 rows x 9 columns]


In [108]:
pd.DataFrame(wins_score, columns = model_names[order_of_models], index=model_names[order_of_models])[['Baseline', 'LPSVS_No_5_5']]

,Baseline,LPSVS_No_5_5
LPSVS_No_5_5,29.0,0.0
LPSVS_No_11_5,28.0,16.0
LPSVS_No_5_1,27.0,14.0
LPSVS_No_11_1,26.0,17.0
LPSVS_100_11_5,21.0,14.0
LPSVS_100_5_5,21.0,13.0
Baseline,0.0,13.0
LPSVS_100_11_1,23.0,13.0
LPSVS_100_5_1,20.0,11.0


In [102]:
model_names[order_of_models]

array(['LPSVS_No_5_5', 'LPSVS_No_11_5', 'LPSVS_No_5_1', 'LPSVS_No_11_1',
       'LPSVS_100_11_5', 'LPSVS_100_5_5', 'Baseline', 'LPSVS_100_11_1',
       'LPSVS_100_5_1'], dtype=object)

In [103]:
print(mean_ranks.reset_index(drop=True).to_string())

            model      rank
0    LPSVS_No_5_5  3.761905
1   LPSVS_No_11_5  4.142857
2    LPSVS_No_5_1  4.357143
3   LPSVS_No_11_1  4.642857
4  LPSVS_100_11_5  5.166667
5   LPSVS_100_5_5  5.380952
6        Baseline  5.642857
7  LPSVS_100_11_1  5.738095
8   LPSVS_100_5_1  6.166667


## Plot performance over number of samples and imbalance ratio for baseline vs best model

#### **No clear trend** when plotting against imbalance, number of samples or number of features

In [95]:
import plotly.express as px

dataset_details = dataset_df[
    # (dataset_df["n_binary_features"] == dataset_df["n_features"])
    (dataset_df["task"] == "classification")
    & (dataset_df["n_classes"] == 2)
    & (dataset_df["n_features"] <= 150)
]


res_to_keep = res[res['model'].isin(["Baseline", "CRH_pc99"])]
res_to_keep['num_samples'] = res_to_keep['dataset'].map(dataset_details[['dataset', 'n_instances']].set_index('dataset')['n_instances'].to_dict().get)
res_to_keep['num_feats'] = res_to_keep['dataset'].map(dataset_details[['dataset', 'n_features']].set_index('dataset')['n_features'].to_dict().get)
# res = pd.DataFrame(res, columns=['dataset', 'dataset_class_imb', 'model', 'time', 'acc', 'pr', 'rec', 'f1'])


fig = px.scatter(res_to_keep, x="dataset_class_imb", y="f1", color="model")
fig.show()

/tmp/ipykernel_200728/2580015051.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_200728/2580015051.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Average time taken

### The overhead is less than a second on average or 10 seconds at most (with 48K samples)

In [109]:
## Average time taken

times = []
for dataset_index, classification_dataset in enumerate(classification_datasets[::-1][:]):
    
    print(f"{classification_dataset} ({dataset_index + 1}/{len(classification_datasets) + 1})")
    X, y = fetch_data(classification_dataset, return_X_y=True)
    if y.max() != 1 or y.min() != 0:
        for wanted, actual in enumerate(np.unique(y)):
            y[y==actual] = wanted
        
    imb_ratio = np.bincount(y).max() / np.bincount(y).min()
    print(f"{X.shape} with ratio : {imb_ratio:.4f}\n")
    
    time_s = time.time()
    scores_data =  np.zeros(len(X)) + max_depth + 1
    for depth in range(1, max_depth):
        clf = DecisionTreeClassifier(random_state=42, max_depth=depth)
        clf.fit(X, y)
        y_pred = clf.predict(X)
        found_flag = (y_pred == y).astype(int)*depth
        found_flag[found_flag == 0] = max_depth + 1
        scores_data = np.minimum(scores_data, found_flag)
        if (scores_data >= max_depth).sum() == 0:
            break
    time_taken = time.time() - time_s
    times.append(time_taken)

xd6 (1/90)
(973, 9) with ratio : 2.0217

wdbc (2/90)
(569, 30) with ratio : 1.6840

vote (3/90)
(435, 16) with ratio : 1.5893

twonorm (4/90)
(7400, 20) with ratio : 1.0016

tokyo1 (5/90)
(959, 44) with ratio : 1.7717

titanic (6/90)
(2099, 8) with ratio : 2.0822

tic_tac_toe (7/90)
(958, 9) with ratio : 1.8855

threeOf9 (8/90)
(512, 9) with ratio : 1.1513

spectf (9/90)
(349, 44) with ratio : 2.6737

spect (10/90)
(267, 22) with ratio : 3.8545

spambase (11/90)
(4601, 57) with ratio : 1.5378

sonar (12/90)
(208, 60) with ratio : 1.1443

saheart (13/90)
(462, 9) with ratio : 1.8875

ring (14/90)
(7400, 20) with ratio : 1.0197

profb (15/90)
(672, 9) with ratio : 2.0000

prnn_synth (16/90)
(250, 2) with ratio : 1.0000

prnn_crabs (17/90)
(200, 7) with ratio : 1.0000

postoperative_patient_data (18/90)
(88, 8) with ratio : 2.6667

pima (19/90)
(768, 8) with ratio : 1.8657

phoneme (20/90)
(5404, 5) with ratio : 2.4073

parity5+5 (21/90)
(1124, 10) with ratio : 1.0180

parity5 (22/90)
(32

In [113]:
print(f"Mean time taken (seconds): {np.mean(times):.2f} +- {2*np.std(times):.2f} (mean +- 2*std)")

Mean time taken (seconds): 0.64 +- 4.02 (mean +- 2*std)
